Focus on exploring the data to find interesting trends, with an overarching goal of answering the following question: “Are soccer referees more likely to give red cards to dark skin toned players than light skin toned players?” Data and a description of the columns are attached.

2012-13 European football (soccer) professional leagues

In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import numpy as np
import scipy as sp
from dateutil.relativedelta import relativedelta

In [15]:
midpoint_end_of_year = pd.to_datetime('2012-12-31')

In [2]:
df = pd.DataFrame.from_csv('redcard.csv')

In [3]:
df.shape

(146028, 27)

In [4]:
df.columns

Index(['player', 'club', 'leagueCountry', 'birthday', 'height', 'weight',
       'position', 'games', 'victories', 'ties', 'defeats', 'goals',
       'yellowCards', 'yellowReds', 'redCards', 'photoID', 'rater1', 'rater2',
       'refNum', 'refCountry', 'Alpha_3', 'meanIAT', 'nIAT', 'seIAT',
       'meanExp', 'nExp', 'seExp'],
      dtype='object')

In [33]:
def calc_age(date2):
    return relativedelta(midpoint_end_of_year, date2).years
df['totalReds'] = df.redCards + df.yellowReds
df['totalCards'] = df.redCards + df.yellowCards
df['birthday'] = pd.to_datetime(df.birthday)
df['Age'] = df.birthday.apply(calc_age)

In [35]:
df['winRate'] = df.victories/df.games
df['tieRate'] = df.ties/df.games
df['lossRate'] = df.defeats/df.games
df['goalRate'] = df.goals/df.games

In [39]:
df.sort_values(by=['games'])

,player,club,leagueCountry,birthday,height,weight,position,games,victories,ties,...,meanExp,nExp,seExp,totalReds,totalCards,Age,winRate,tieRate,lossRate,goalRate
playerShort,,,,,,,,,,,,,,,,,,,,,
lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,1983-08-31,177.0,72.0,Attacking Midfielder,1,0,0,...,0.396000,750.0,0.002696,0,0,29,0.000000,0.000000,1.000000,0.000000
mauro-boselli,Mauro Boselli,Wigan Athletic,England,1985-05-22,175.0,73.0,Center Forward,1,0,1,...,0.697417,271.0,0.008707,0,0,27,0.000000,1.000000,0.000000,1.000000
javier-mascherano,Javier Mascherano,FC Barcelona,Spain,1984-08-06,174.0,74.0,Defensive Midfielder,1,1,0,...,0.697417,271.0,0.008707,0,0,28,1.000000,0.000000,0.000000,0.000000
carlos-tevez,Carlos Tévez,Manchester City,England,1984-05-02,171.0,75.0,Left Winger,1,1,0,...,0.697417,271.0,0.008707,0,0,28,1.000000,0.000000,0.000000,1.000000
alexis-sanchez,Alexis Sánchez,FC Barcelona,Spain,1988-12-19,170.0,73.0,Right Winger,1,1,0,...,0.697417,271.0,0.008707,0,0,24,1.000000,0.000000,0.000000,0.000000
xavi,Xavi,FC Barcelona,Spain,1980-01-25,170.0,68.0,Attacking Midfielder,1,0,0,...,0.588297,1897.0,0.001002,0,0,32,0.000000,0.000000,1.000000,0.000000
william-gallas,William Gallas,Tottenham Hotspur,England,1977-08-17,181.0,72.0,Center Back,1,0,0,...,0.588297,1897.0,0.001002,0,0,35,0.000000,0.000000,1.000000,0.000000
tomas-rosicky,Tomáš Rosický,Arsenal FC,England,1980-04-10,178.0,67.0,Attacking Midfielder,1,0,1,...,0.588297,1897.0,0.001002,0,0,32,0.000000,1.000000,0.000000,0.000000
steed-malbranque,Steed Malbranque,Olympique Lyon,France,1980-06-01,173.0,72.0,Attacking Midfielder,1,1,0,...,0.588297,1897.0,0.001002,0,0,32,1.000000,0.000000,0.000000,0.000000


In [30]:
# print(df.position.unique())
# print(df.Alpha_3.unique())
# print(df.birthday.dt.year)